In [5]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [6]:
from pyod.models.hbos import HBOS
from pyod.utils.data import generate_data
def main_task(contamination, n_bins):

    model = HBOS(contamination=contamination, n_bins=n_bins)
    model.fit(X_train)

    # 이상치 점수 계산
    y_scores = model.decision_function(X_test)

    # 이상치 예측
    y_pred = model.predict(X_test)

    return round(f1_score(y_test, y_pred),3)

In [10]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    contamination = trial.suggest_float('contamination', 0.01, 0.5, step = 0.02)
    n_bins = trial.suggest_int('n_bins', 10, 100, step = 2)
    print('contamination : ', contamination)
    print('n_bins : ', n_bins)

    f1_score = main_task(contamination=contamination, n_bins = n_bins)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=300)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 14:12:47,619] A new study created in memory with name: no-name-fb6366cd-e04c-4b59-ab5d-ed6b3aec5b60
[I 2023-11-08 14:12:47,639] Trial 0 finished with value: 0.736 and parameters: {'contamination': 0.37, 'n_bins': 86}. Best is trial 0 with value: 0.736.
[I 2023-11-08 14:12:47,656] Trial 1 finished with value: 0.74 and parameters: {'contamination': 0.31, 'n_bins': 34}. Best is trial 1 with value: 0.74.
[I 2023-11-08 14:12:47,674] Trial 2 finished with value: 0.745 and parameters: {'contamination': 0.47000000000000003, 'n_bins': 38}. Best is trial 2 with value: 0.745.
[I 2023-11-08 14:12:47,690] Trial 3 finished with value: 0.751 and parameters: {'contamination': 0.47000000000000003, 'n_bins': 48}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,707] Trial 4 finished with value: 0.746 and parameters: {'contamination': 0.31, 'n_bins': 70}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,727] Trial 5 finished with value: 0.733 and parameters: {'contaminatio

contamination :  0.37
n_bins :  86
<f1_score: 0.736 >
contamination :  0.31
n_bins :  34
<f1_score: 0.74 >
contamination :  0.47000000000000003
n_bins :  38
<f1_score: 0.745 >
contamination :  0.47000000000000003
n_bins :  48
<f1_score: 0.751 >
contamination :  0.31
n_bins :  70
<f1_score: 0.746 >
contamination :  0.27
n_bins :  44
<f1_score: 0.733 >
contamination :  0.27
n_bins :  96
<f1_score: 0.748 >
contamination :  0.15000000000000002
n_bins :  66
<f1_score: 0.631 >
contamination :  0.13
n_bins :  30
<f1_score: 0.602 >
contamination :  0.45
n_bins :  50
<f1_score: 0.745 >
contamination :  0.03
n_bins :  14
<f1_score: 0.289 >


[I 2023-11-08 14:12:47,827] Trial 10 finished with value: 0.289 and parameters: {'contamination': 0.03, 'n_bins': 14}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,856] Trial 11 finished with value: 0.685 and parameters: {'contamination': 0.17, 'n_bins': 98}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,886] Trial 12 finished with value: 0.751 and parameters: {'contamination': 0.41000000000000003, 'n_bins': 68}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,936] Trial 13 finished with value: 0.751 and parameters: {'contamination': 0.41000000000000003, 'n_bins': 66}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,961] Trial 14 finished with value: 0.743 and parameters: {'contamination': 0.49, 'n_bins': 80}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:47,982] Trial 15 finished with value: 0.749 and parameters: {'contamination': 0.37, 'n_bins': 56}. Best is trial 3 with value: 0.751.
[I 2023-11-08 14:12:48,009] Trial 16 finished 

contamination :  0.17
n_bins :  98
<f1_score: 0.685 >
contamination :  0.41000000000000003
n_bins :  68
<f1_score: 0.751 >
contamination :  0.41000000000000003
n_bins :  66
<f1_score: 0.751 >
contamination :  0.49
n_bins :  80
<f1_score: 0.743 >
contamination :  0.37
n_bins :  56
<f1_score: 0.749 >
contamination :  0.41000000000000003
n_bins :  22
<f1_score: 0.747 >
contamination :  0.35000000000000003
n_bins :  58
<f1_score: 0.752 >
contamination :  0.35000000000000003
n_bins :  54


[I 2023-11-08 14:12:48,063] Trial 18 finished with value: 0.752 and parameters: {'contamination': 0.35000000000000003, 'n_bins': 54}. Best is trial 17 with value: 0.752.
[I 2023-11-08 14:12:48,093] Trial 19 finished with value: 0.683 and parameters: {'contamination': 0.21000000000000002, 'n_bins': 58}. Best is trial 17 with value: 0.752.
[I 2023-11-08 14:12:48,112] Trial 20 finished with value: 0.743 and parameters: {'contamination': 0.35000000000000003, 'n_bins': 80}. Best is trial 17 with value: 0.752.
[I 2023-11-08 14:12:48,136] Trial 21 finished with value: 0.747 and parameters: {'contamination': 0.33, 'n_bins': 48}. Best is trial 17 with value: 0.752.
[I 2023-11-08 14:12:48,163] Trial 22 finished with value: 0.753 and parameters: {'contamination': 0.43, 'n_bins': 42}. Best is trial 22 with value: 0.753.
[I 2023-11-08 14:12:48,187] Trial 23 finished with value: 0.75 and parameters: {'contamination': 0.41000000000000003, 'n_bins': 40}. Best is trial 22 with value: 0.753.
[I 2023-11-

...

<f1_score: 0.776 >
contamination :  0.23
n_bins :  16
<f1_score: 0.765 >
contamination :  0.21000000000000002
n_bins :  12
<f1_score: 0.761 >
contamination :  0.25
n_bins :  18
<f1_score: 0.772 >
contamination :  0.23
n_bins :  10
<f1_score: 0.78 >
contamination :  0.21000000000000002
n_bins :  14
<f1_score: 0.781 >
contamination :  0.13
n_bins :  16
<f1_score: 0.646 >


[I 2023-11-08 14:12:55,964] Trial 288 finished with value: 0.782 and parameters: {'contamination': 0.23, 'n_bins': 14}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:55,999] Trial 289 finished with value: 0.735 and parameters: {'contamination': 0.27, 'n_bins': 56}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,034] Trial 290 finished with value: 0.75 and parameters: {'contamination': 0.23, 'n_bins': 18}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,064] Trial 291 finished with value: 0.716 and parameters: {'contamination': 0.25, 'n_bins': 50}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,097] Trial 292 finished with value: 0.758 and parameters: {'contamination': 0.23, 'n_bins': 12}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,125] Trial 293 finished with value: 0.765 and parameters: {'contamination': 0.25, 'n_bins': 16}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,152] Trial 294 finished with value: 

contamination :  0.23
n_bins :  14
<f1_score: 0.782 >
contamination :  0.27
n_bins :  56
<f1_score: 0.735 >
contamination :  0.23
n_bins :  18
<f1_score: 0.75 >
contamination :  0.25
n_bins :  50
<f1_score: 0.716 >
contamination :  0.23
n_bins :  12
<f1_score: 0.758 >
contamination :  0.25
n_bins :  16
<f1_score: 0.765 >
contamination :  0.27
n_bins :  12
<f1_score: 0.763 >


[I 2023-11-08 14:12:56,188] Trial 295 finished with value: 0.782 and parameters: {'contamination': 0.23, 'n_bins': 14}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,220] Trial 296 finished with value: 0.78 and parameters: {'contamination': 0.23, 'n_bins': 10}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,250] Trial 297 finished with value: 0.75 and parameters: {'contamination': 0.23, 'n_bins': 18}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,284] Trial 298 finished with value: 0.776 and parameters: {'contamination': 0.25, 'n_bins': 14}. Best is trial 117 with value: 0.801.
[I 2023-11-08 14:12:56,313] Trial 299 finished with value: 0.765 and parameters: {'contamination': 0.25, 'n_bins': 16}. Best is trial 117 with value: 0.801.


contamination :  0.23
n_bins :  14
<f1_score: 0.782 >
contamination :  0.23
n_bins :  10
<f1_score: 0.78 >
contamination :  0.23
n_bins :  18
<f1_score: 0.75 >
contamination :  0.25
n_bins :  14
<f1_score: 0.776 >
contamination :  0.25
n_bins :  16
<f1_score: 0.765 >
study.best_trial.value : 0.801
study.best_params : {'contamination': 0.15000000000000002, 'n_bins': 10}


In [11]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.801
study.best_paramse : {'contamination': 0.15000000000000002, 'n_bins': 10}
